In [1]:
import numpy as np
from scipy.optimize import minimize
#import scipy.io
import matplotlib.pyplot as plt

## Código inicial 

In [2]:
#formato binario
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import minimize
import gspread
from oauth2client.service_account import ServiceAccountCredentials

In [3]:
def obtener_datos():
    # use creds to create a client to interact with the Google Drive API
    scope = ['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/drive']
    creds = ServiceAccountCredentials.from_json_keyfile_name('client_secret.json', scope)
    client = gspread.authorize(creds)
    # Find a workbook by name and open the first sheet
    # Make sure you use the right name here.
    sheet = client.open("DatosTFG_SistemasRecomendacion").sheet1

    # Extract and print all of the values
    list_of_hashes = sheet.get_all_records()
    #print(list_of_hashes)
    tabla = pd.DataFrame(data=sheet.get_all_records())
    return tabla

In [4]:
tabla=obtener_datos()

## Crear matrices Y, R
Filas: ASIGNATURAS
Columnas: USUARIOS

In [5]:
# Eliminar columnas innecesarias
tabla2=tabla.drop(columns=['Submitted At', 'Token']).T
# Crear matriz de ratings Y
Y=tabla2.values
# Convertir espacios en ceros
Y[Y=='']=0 # eliminar los espacios
# Crear matriz R
R=np.zeros_like(Y)
R[Y>0]=1

In [6]:
Y.shape

(46, 56)

In [8]:
# revisar por una columna (usuario) aleatorio
u=np.random.randint(Y.shape[1])
print('usuario %i\n' %u)
column=tabla2[u]
for j in range(Y.shape[0]):
    print(column[j],Y[j,u],R[j,u],column.index[j])

usuario 40

5 5 1 Algoritmia
3 3 1 Análisis y Diseño de Sistemas
3 3 1 Aplicaciones de Bases de Datos
4 4 1 Arquitectura de Computadores
3 3 1 Arquitecturas Paralelas
3 3 1 Bases de Datos
3 3 1 Computación Neuronal y Evolutiva
3 3 1 Control por Computador
3 3 1 Cálculo
3 3 1 Desarrollo Avanzado de Sistemas Software
3 3 1 Diseño e Implementación de Sistemas Digitales
4 4 1 Diseño y Administración de Sistemas y Redes 
4 4 1 Diseño y Mantenimiento del Software
4 4 1 Estadística
4 4 1 Estructuras de Datos
2 2 1 Fundamentos Deontológicos y Jurídicos de las TIC
3 3 1 Fundamentos Físicos de la Informática
4 4 1 Fundamentos de Computadores
3 3 1 Fundamentos de Organización y Gestión de Empresas
4 4 1 Gestión de Proyectos
4 4 1 Gestión de la Información
5 5 1 Hardware de Aplicación Específica
3 3 1 Informática Básica
3 3 1 Ingeniería del Software
2 2 1 Inglés Aplicado a la Informática
3 3 1 Interacción Hombre-Máquina
5 5 1 Mantenimiento de Equipos Informáticos
3 3 1 Matemática Discreta
3 3 1 Me

## Ajujste del filtro

In [10]:

def cofi_cost_funct(parameters, Y, R, n_users,n_items, n_features, lamb):
    cost =0
    gradient = np.zeros_like(parameters)
    # Si pasamos R como argumento, esto sobra 
    #R = np.zeros_like(Y)
    #R[Y != 0] = 1
    
    Theta= np.reshape(parameters[0: n_users* n_features], (n_users, n_features))
    X= np.reshape(parameters[n_users*n_features:] , (n_items,n_features))
    
    # Para reducir el número de multiplicaciones matriciales
    error=(X.dot(Theta.T)- Y )*R
    
    J= (1/2)*sum(sum(np.power(error,2)))
    cost= J + ((lamb/2) * sum(sum(np.power(X,2))))+((lamb/2) * sum(sum(np.power(Theta, 2))))
    X_gradient = error.dot(Theta) + X*lamb               
    Theta_gradient = error.T.dot(X) + Theta*lamb
    
    # Hay que pasar los gradientes (que son matrices) a vectores
    Theta_gradient_vector=np.reshape(Theta_gradient,n_users*n_features)
    X_gradient_vector=np.reshape(X_gradient,n_items*n_features)    
    
    gradient=np.concatenate((Theta_gradient_vector,X_gradient_vector))
    #print(cost, gradient)
    return (cost, gradient)

In [11]:
# Get dimensions: n_users and n_movies
# Your code ..............
n_users=Y.shape[1]
n_items=Y.shape[0]


In [12]:
# Set the number of features
n_features = 100

In [13]:
# Set the initial parameters (Theta, X) with random values
# Your code ..............
X=np.random.random((n_items,n_features))
Theta=np.random.random((n_users,n_features))


In [14]:
# fold X and Theta into the variable initial_parameters
# Your code ..............
initial_parameters = np.append(Theta.flatten(),X.flatten())

In [15]:
# Set the regularization parameter
lamb = 10

# Define a function to be minimized
def cofiCostFunc_minimize(parameters):
    return cofi_cost_funct(parameters,Y, R, n_users, n_items, n_features,lamb)

# Set the number of iteations
max_iter=200

In [16]:
# Minimize the function using minimize from the package scipy.optimize and get the optimized parameters
parameters = (minimize(cofiCostFunc_minimize,initial_parameters,method="CG",jac=True,
                   options={'maxiter':max_iter, "disp":True})).x

Optimization terminated successfully.
         Current function value: 2582.857977
         Iterations: 183
         Function evaluations: 284
         Gradient evaluations: 284


In [18]:
# Unfold the returned optimized parameters back into X and Theta
# Your code ..............
Theta=parameters[0:n_users*n_features].reshape(n_users,n_features)
X=parameters[n_users*n_features:].reshape(n_items,n_features)
predictions = np.dot(X,Theta.T)

In [35]:
# Predicción para un usuario aleatorio
u=np.random.randint(Y.shape[1])
print('Usuario %i' %u)
p=predictions[:,u]
recommendations=[]
asignaturas=tabla2.index
print('Predicción\t Asignatura')
for i in range(5):
    maxp= np.max(p)
    [rm]=np.where(p==maxp)
    recommendations.append(asignaturas[rm[0]])
    p[rm[0]]=0
    print('(%0.4f)\t' % (maxp),asignaturas[rm[0]])

Usuario 19
Predicción	 Asignatura
(4.4563)	 Bases de Datos
(4.3653)	 Aplicaciones de Bases de Datos
(4.2727)	 Hardware de Aplicación Específica
(4.2182)	 Sistemas Inteligentes
(4.1794)	 Metodología de la Programación
